# Leitura de bibliotecas e dados necessários

In [389]:
# Importando as bibliotecas
!pip install pandas oracledb numpy
!pip install python-dotenv
from google.colab import drive
import pandas as pd
import numpy as np
import oracledb
import os
from dotenv import load_dotenv
from datetime import datetime
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [391]:
# Lendo os arquivos necessários para o desenvolvimento da tarefa
load_dotenv()

CREDENTIALS = {
'user' : os.getenv('ORACLE_USER'),
'password' : os.getenv('ORACLE_PASSWORD'),
'dsn' : os.getenv('ORACLE_DSN')
}

df_selic = pd.read_csv("/content/drive/MyDrive/FIAP/Selic.csv", sep = ";")
df_ipca = pd.read_csv("/content/drive/MyDrive/FIAP/IPCA.csv", sep = ";")
df_pib_uf = pd.read_csv("/content/drive/MyDrive/FIAP/PIB_UF.csv", sep = ";", skiprows=3)
df_pmc = pd.read_csv("/content/drive/MyDrive/FIAP/PMC.csv", sep=',', decimal='.', skiprows=4)

In [392]:
estados_uf = {
        'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
        'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
        'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
        'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
        'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
        'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
        'Sergipe': 'SE', 'Tocantins': 'TO'
}

meses = {
        'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04', 'maio': '05', 'junho': '06',
        'julho': '07', 'agosto': '08', 'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
}

## Selic

In [393]:
df_selic.head()

,Date,11 - Interest rate - Selic - % p.d.
0,02/01/2023,0.050788
1,03/01/2023,0.050788
2,04/01/2023,0.050788
3,05/01/2023,0.050788
4,06/01/2023,0.050788


In [394]:
df_selic.tail()

,Date,11 - Interest rate - Selic - % p.d.
744,17/12/2025,0.055131
745,18/12/2025,0.055131
746,19/12/2025,0.055131
747,22/12/2025,0.055131
748,Source,BCB-Demab


In [395]:
df_selic.drop(df_selic.index[-1], inplace = True)
df_selic.tail()

,Date,11 - Interest rate - Selic - % p.d.
743,16/12/2025,0.055131
744,17/12/2025,0.055131
745,18/12/2025,0.055131
746,19/12/2025,0.055131
747,22/12/2025,0.055131


In [396]:
df_selic.dtypes

,0
Date,object
11 - Interest rate - Selic - % p.d.,object


In [397]:
df_selic["Date"] = pd.to_datetime(df_selic["Date"], format = '%d/%m/%Y', errors = "coerce")
df_selic.rename(columns={'11 - Interest rate - Selic - % p.d.': 'Taxa'}, inplace=True)
df_selic["Taxa"] = df_selic["Taxa"].astype("float")
df_selic.head()

,Date,Taxa
0,2023-01-02,0.050788
1,2023-01-03,0.050788
2,2023-01-04,0.050788
3,2023-01-05,0.050788
4,2023-01-06,0.050788


In [398]:
df_selic["UF"] = "BR"
df_selic

,Date,Taxa,UF
0,2023-01-02,0.050788,BR
1,2023-01-03,0.050788,BR
2,2023-01-04,0.050788,BR
3,2023-01-05,0.050788,BR
4,2023-01-06,0.050788,BR
...,...,...,...
743,2025-12-16,0.055131,BR
744,2025-12-17,0.055131,BR
745,2025-12-18,0.055131,BR
746,2025-12-19,0.055131,BR


## PIB UF

In [399]:
df_pib_uf.head()

,Nível,Cód.,Unidade da Federação,2019,2020,2021,2022,2023
0,UF,11,Rondônia,47091336,51598741,58170096,66795454,76456179
1,UF,12,Acre,15630017,16476371,21374440,23676136,26291321
2,UF,13,Amazonas,108181091,116019139,131531038,145140465,161794976
3,UF,14,Roraima,14292227,16024276,18202579,21095342,25124805
4,UF,15,Pará,178376984,215935604,262904979,236141874,254546511


In [400]:
df_pib_uf = df_pib_uf.drop(["Nível", "Cód."], axis = 1)
df_pib_uf.head()

,Unidade da Federação,2019,2020,2021,2022,2023
0,Rondônia,47091336,51598741,58170096,66795454,76456179
1,Acre,15630017,16476371,21374440,23676136,26291321
2,Amazonas,108181091,116019139,131531038,145140465,161794976
3,Roraima,14292227,16024276,18202579,21095342,25124805
4,Pará,178376984,215935604,262904979,236141874,254546511


In [401]:
cols_anos = [c for c in df_pib_uf.columns if c.isdigit()]
df_pib_uf_melt = df_pib_uf.melt(
    id_vars = ["Unidade da Federação"],
    value_vars = cols_anos,
    var_name = "Ano",
    value_name = "Indicador"
)
df_pib_uf_melt.head()

,Unidade da Federação,Ano,Indicador
0,Rondônia,2019,47091336
1,Acre,2019,15630017
2,Amazonas,2019,108181091
3,Roraima,2019,14292227
4,Pará,2019,178376984


In [402]:
df_pib_uf_melt.dtypes

,0
Unidade da Federação,object
Ano,object
Indicador,int64


In [403]:
df_pib_uf_melt['Unidade da Federação'] = df_pib_uf_melt['Unidade da Federação'].map(estados_uf)
df_pib_uf_melt

,Unidade da Federação,Ano,Indicador
0,RO,2019,47091336
1,AC,2019,15630017
2,AM,2019,108181091
3,RR,2019,14292227
4,PA,2019,178376984
...,...,...,...
130,RS,2023,650107022
131,MS,2023,184402118
132,MT,2023,273008586
133,GO,2023,336746975


In [404]:
df_pib_uf_melt["Ano"] = pd.to_datetime(df_pib_uf_melt["Ano"], format = '%Y', errors = "coerce")
df_pib_uf_melt.head()

,Unidade da Federação,Ano,Indicador
0,RO,2019-01-01,47091336
1,AC,2019-01-01,15630017
2,AM,2019-01-01,108181091
3,RR,2019-01-01,14292227
4,PA,2019-01-01,178376984


In [405]:
df_pib_uf_melt.tail()

,Unidade da Federação,Ano,Indicador
130,RS,2023-01-01,650107022
131,MS,2023-01-01,184402118
132,MT,2023-01-01,273008586
133,GO,2023-01-01,336746975
134,DF,2023-01-01,365669108


## IPCA

In [406]:
df_ipca.head()

,Date,433 - Broad National Consumer Price Index (IPCA) - Monthly % var.
0,01/2023,0.53
1,02/2023,0.84
2,03/2023,0.71
3,04/2023,0.61
4,05/2023,0.23


In [407]:
df_ipca.rename(columns={"433 - Broad National Consumer Price Index (IPCA) - Monthly % var.": "Taxa"}, inplace=True)
df_ipca.head()

,Date,Taxa
0,01/2023,0.53
1,02/2023,0.84
2,03/2023,0.71
3,04/2023,0.61
4,05/2023,0.23


In [408]:
df_ipca.tail()

,Date,Taxa
31,08/2025,-0.11
32,09/2025,0.48
33,10/2025,0.09
34,11/2025,0.18
35,Source,IBGE


In [409]:
df_ipca.drop(df_ipca.index[-1], inplace = True)
df_ipca.tail()

,Date,Taxa
30,07/2025,0.26
31,08/2025,-0.11
32,09/2025,0.48
33,10/2025,0.09
34,11/2025,0.18


In [410]:
df_ipca.dtypes

,0
Date,object
Taxa,object


In [411]:
df_ipca["Date"] = pd.to_datetime(df_ipca["Date"], format = '%m/%Y', errors = "coerce")
df_ipca["Taxa"] = df_ipca["Taxa"].astype("float")
df_ipca["UF"] = "BR"
df_ipca.head()

,Date,Taxa,UF
0,2023-01-01,0.53,BR
1,2023-02-01,0.84,BR
2,2023-03-01,0.71,BR
3,2023-04-01,0.61,BR
4,2023-05-01,0.23,BR


In [412]:
df_ipca.dtypes

,0
Date,datetime64[ns]
Taxa,float64
UF,object


## PMCA

In [413]:
df_pmc.head()

,Nível,Unidade da Federação,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023,agosto 2023,...,janeiro 2025,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,outubro 2025
0,UF,Rondônia,10.1,6.6,6.0,4.5,3.1,2.2,1.9,1.6,...,4.5,8.0,8.9,10.2,10.4,10.0,9.4,8.6,8.5,8.6
1,UF,Acre,14.1,13.1,12.3,10.6,9.9,10.1,10.0,9.6,...,8.5,8.3,8.5,8.5,8.8,8.6,8.4,8.4,8.2,7.9
2,UF,Amazonas,5.7,5.1,6.8,6.1,5.2,4.8,4.6,4.5,...,8.2,11.0,8.3,8.1,7.6,7.4,7.0,6.7,6.3,5.9
3,UF,Roraima,13.6,13.4,12.3,11.5,9.3,7.7,7.0,5.9,...,0.8,1.8,2.1,1.9,3.1,3.8,4.7,4.6,4.1,3.1
4,UF,Pará,6.0,5.1,4.9,4.4,4.0,3.5,3.2,2.9,...,7.9,5.1,5.2,7.0,7.6,7.4,7.4,7.1,6.7,6.4


In [414]:
df_pmc = df_pmc.drop("Nível", axis = 1)
df_pmc.head()

,Unidade da Federação,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023,agosto 2023,setembro 2023,...,janeiro 2025,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,outubro 2025
0,Rondônia,10.1,6.6,6.0,4.5,3.1,2.2,1.9,1.6,1.9,...,4.5,8.0,8.9,10.2,10.4,10.0,9.4,8.6,8.5,8.6
1,Acre,14.1,13.1,12.3,10.6,9.9,10.1,10.0,9.6,9.2,...,8.5,8.3,8.5,8.5,8.8,8.6,8.4,8.4,8.2,7.9
2,Amazonas,5.7,5.1,6.8,6.1,5.2,4.8,4.6,4.5,4.6,...,8.2,11.0,8.3,8.1,7.6,7.4,7.0,6.7,6.3,5.9
3,Roraima,13.6,13.4,12.3,11.5,9.3,7.7,7.0,5.9,5.2,...,0.8,1.8,2.1,1.9,3.1,3.8,4.7,4.6,4.1,3.1
4,Pará,6.0,5.1,4.9,4.4,4.0,3.5,3.2,2.9,3.0,...,7.9,5.1,5.2,7.0,7.6,7.4,7.4,7.1,6.7,6.4


In [415]:
df_pmc.tail()

,Unidade da Federação,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023,agosto 2023,setembro 2023,...,janeiro 2025,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,outubro 2025
32,Zero resultante de um cálculo ou arredondament...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,Valor inibido para não identificar o informant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Valor não se aplica.\nEx: Não se pode obter o ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Valor não disponível.\nEx: A produção de feijã...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Significa uma faixa de valores. Varia em funçã...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [416]:
df_pmc

,Unidade da Federação,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023,agosto 2023,setembro 2023,...,janeiro 2025,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,outubro 2025
0,Rondônia,10.1,6.6,6.0,4.5,3.1,2.2,1.9,1.6,1.9,...,4.5,8.0,8.9,10.2,10.4,10.0,9.4,8.6,8.5,8.6
1,Acre,14.1,13.1,12.3,10.6,9.9,10.1,10.0,9.6,9.2,...,8.5,8.3,8.5,8.5,8.8,8.6,8.4,8.4,8.2,7.9
2,Amazonas,5.7,5.1,6.8,6.1,5.2,4.8,4.6,4.5,4.6,...,8.2,11.0,8.3,8.1,7.6,7.4,7.0,6.7,6.3,5.9
3,Roraima,13.6,13.4,12.3,11.5,9.3,7.7,7.0,5.9,5.2,...,0.8,1.8,2.1,1.9,3.1,3.8,4.7,4.6,4.1,3.1
4,Pará,6.0,5.1,4.9,4.4,4.0,3.5,3.2,2.9,3.0,...,7.9,5.1,5.2,7.0,7.6,7.4,7.4,7.1,6.7,6.4
5,Amapá,12.4,9.1,8.9,7.4,5.7,4.2,3.2,2.2,1.7,...,21.8,20.0,19.0,17.9,16.6,15.2,15.3,14.3,14.5,14.6
6,Tocantins,19.5,19.9,19.6,18.4,18.3,17.0,16.2,15.5,15.0,...,13.9,12.1,9.4,8.6,6.7,5.3,3.2,2.3,2.4,2.5
7,Maranhão,20.7,18.3,16.2,14.2,13.5,12.2,11.3,10.4,10.3,...,7.3,8.9,7.9,8.6,7.8,7.5,7.3,7.6,7.5,7.2
8,Piauí,13.1,10.1,9.4,7.1,5.0,3.4,2.9,2.4,2.3,...,7.5,7.7,8.9,9.1,9.0,7.9,7.1,6.2,5.4,4.8
9,Ceará,18.5,16.0,15.7,13.6,12.3,11.9,12.1,11.9,12.4,...,9.0,9.8,9.0,9.5,9.1,8.7,8.5,8.3,8.5,8.5


In [417]:
df_pmc = df_pmc[df_pmc.iloc[:,0].isin(estados_uf.keys())]
df_pmc

,Unidade da Federação,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023,agosto 2023,setembro 2023,...,janeiro 2025,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,outubro 2025
0,Rondônia,10.1,6.6,6.0,4.5,3.1,2.2,1.9,1.6,1.9,...,4.5,8.0,8.9,10.2,10.4,10.0,9.4,8.6,8.5,8.6
1,Acre,14.1,13.1,12.3,10.6,9.9,10.1,10.0,9.6,9.2,...,8.5,8.3,8.5,8.5,8.8,8.6,8.4,8.4,8.2,7.9
2,Amazonas,5.7,5.1,6.8,6.1,5.2,4.8,4.6,4.5,4.6,...,8.2,11.0,8.3,8.1,7.6,7.4,7.0,6.7,6.3,5.9
3,Roraima,13.6,13.4,12.3,11.5,9.3,7.7,7.0,5.9,5.2,...,0.8,1.8,2.1,1.9,3.1,3.8,4.7,4.6,4.1,3.1
4,Pará,6.0,5.1,4.9,4.4,4.0,3.5,3.2,2.9,3.0,...,7.9,5.1,5.2,7.0,7.6,7.4,7.4,7.1,6.7,6.4
5,Amapá,12.4,9.1,8.9,7.4,5.7,4.2,3.2,2.2,1.7,...,21.8,20.0,19.0,17.9,16.6,15.2,15.3,14.3,14.5,14.6
6,Tocantins,19.5,19.9,19.6,18.4,18.3,17.0,16.2,15.5,15.0,...,13.9,12.1,9.4,8.6,6.7,5.3,3.2,2.3,2.4,2.5
7,Maranhão,20.7,18.3,16.2,14.2,13.5,12.2,11.3,10.4,10.3,...,7.3,8.9,7.9,8.6,7.8,7.5,7.3,7.6,7.5,7.2
8,Piauí,13.1,10.1,9.4,7.1,5.0,3.4,2.9,2.4,2.3,...,7.5,7.7,8.9,9.1,9.0,7.9,7.1,6.2,5.4,4.8
9,Ceará,18.5,16.0,15.7,13.6,12.3,11.9,12.1,11.9,12.4,...,9.0,9.8,9.0,9.5,9.1,8.7,8.5,8.3,8.5,8.5


In [418]:
df_pmc_melt = df_pmc.melt(
    id_vars = [df_pmc.columns[0]],
    var_name = "data_txt",
    value_name = "vl_indicador"
)
df_pmc_melt

,Unidade da Federação,data_txt,vl_indicador
0,Rondônia,janeiro 2023,10.1
1,Acre,janeiro 2023,14.1
2,Amazonas,janeiro 2023,5.7
3,Roraima,janeiro 2023,13.6
4,Pará,janeiro 2023,6.0
...,...,...,...
913,Rio Grande do Sul,outubro 2025,6.8
914,Mato Grosso do Sul,outubro 2025,5.7
915,Mato Grosso,outubro 2025,8.4
916,Goiás,outubro 2025,6.4


In [419]:
df_pmc_melt['Unidade da Federação'] = df_pmc_melt['Unidade da Federação'].map(estados_uf)
df_pmc_melt

,Unidade da Federação,data_txt,vl_indicador
0,RO,janeiro 2023,10.1
1,AC,janeiro 2023,14.1
2,AM,janeiro 2023,5.7
3,RR,janeiro 2023,13.6
4,PA,janeiro 2023,6.0
...,...,...,...
913,RS,outubro 2025,6.8
914,MS,outubro 2025,5.7
915,MT,outubro 2025,8.4
916,GO,outubro 2025,6.4


In [420]:
df_pmc_melt.dtypes

,0
Unidade da Federação,object
data_txt,object
vl_indicador,float64


In [421]:
def converte_data_pt(texto):
  try:
    mes_txt, ano = texto.split(' ')
    mes_num = meses.get(mes_txt.lower())
    return pd.to_datetime(f"{ano}-{mes_num}-01") + pd.offsets.MonthEnd(0)
  except:
    return None
df_pmc_melt['date'] = df_pmc_melt['data_txt'].apply(converte_data_pt)

In [422]:
df_pmc_melt.head()

,Unidade da Federação,data_txt,vl_indicador,date
0,RO,janeiro 2023,10.1,2023-01-31
1,AC,janeiro 2023,14.1,2023-01-31
2,AM,janeiro 2023,5.7,2023-01-31
3,RR,janeiro 2023,13.6,2023-01-31
4,PA,janeiro 2023,6.0,2023-01-31


In [423]:
df_pmc_melt = df_pmc_melt.drop('data_txt', axis =1)
df_pmc_melt

,Unidade da Federação,vl_indicador,date
0,RO,10.1,2023-01-31
1,AC,14.1,2023-01-31
2,AM,5.7,2023-01-31
3,RR,13.6,2023-01-31
4,PA,6.0,2023-01-31
...,...,...,...
913,RS,6.8,2025-10-31
914,MS,5.7,2025-10-31
915,MT,8.4,2025-10-31
916,GO,6.4,2025-10-31


## Alimentando o SQL

In [424]:
df_selic["Nome_Indicador"] = "Selic"
df_selic.head()

,Date,Taxa,UF,Nome_Indicador
0,2023-01-02,0.050788,BR,Selic
1,2023-01-03,0.050788,BR,Selic
2,2023-01-04,0.050788,BR,Selic
3,2023-01-05,0.050788,BR,Selic
4,2023-01-06,0.050788,BR,Selic


In [425]:
df_selic_final = df_selic[["UF", "Date", "Nome_Indicador", "Taxa"]].copy()

In [426]:
df_pib_uf_melt["Nome_Indicador"] = "PIB"
df_pib_uf_melt = df_pib_uf_melt.rename(columns = {"Unidade da Federação": "UF", "Ano": "Date"})
df_pib_uf_melt.head()

,UF,Date,Indicador,Nome_Indicador
0,RO,2019-01-01,47091336,PIB
1,AC,2019-01-01,15630017,PIB
2,AM,2019-01-01,108181091,PIB
3,RR,2019-01-01,14292227,PIB
4,PA,2019-01-01,178376984,PIB


In [427]:
df_pib_uf_final = df_pib_uf_melt[["UF", "Date", "Nome_Indicador", "Indicador"]].copy()

In [428]:
df_ipca["Nome_Indicador"] = "IPCA"
df_ipca.head()

,Date,Taxa,UF,Nome_Indicador
0,2023-01-01,0.53,BR,IPCA
1,2023-02-01,0.84,BR,IPCA
2,2023-03-01,0.71,BR,IPCA
3,2023-04-01,0.61,BR,IPCA
4,2023-05-01,0.23,BR,IPCA


In [429]:
df_ipca_final = df_ipca[["UF", "Date", "Nome_Indicador", "Taxa"]].copy()

In [430]:
df_pmc_melt = df_pmc_melt.rename(columns = {"Unidade da Federação": "UF", "vl_indicador": "Indicador", "date": "Date"})
df_pmc_melt["Nome_Indicador"] = "PMC"
df_pmc_melt.head()

,UF,Indicador,Date,Nome_Indicador
0,RO,10.1,2023-01-31,PMC
1,AC,14.1,2023-01-31,PMC
2,AM,5.7,2023-01-31,PMC
3,RR,13.6,2023-01-31,PMC
4,PA,6.0,2023-01-31,PMC


In [431]:
df_pmc_melt_final = df_pmc_melt[["UF", "Date", "Nome_Indicador", "Indicador"]].copy()

In [432]:
# Código para alimentação dos dados Nuclea no Banco de Dados Oracle
USER = CREDENTIALS['user']
PASSWORD = CREDENTIALS["password"]
DSN = CREDENTIALS["dsn"]  # Exemplo de DSN

# Criando a conexão
connection = None
try:
    with oracledb.connect(user=USER, password=PASSWORD, dsn=DSN) as connection:
      cursor = connection.cursor()
      print("Conectou")

      dados_selic = df_selic_final.values.tolist()
      sql_insert_macro = """
          INSERT INTO T_BF_MACRO_ECONOMIA (sg_uf, dt_referencia, nm_indicador, vl_indicador)
          VALUES (:1, :2, :3, :4)
      """
      cursor.executemany(sql_insert_macro, dados_selic)
      print(f"   -> {cursor.rowcount} selic inserido.")

      dados_pib = df_pib_uf_final.values.tolist()
      sql_insert_macro = """
          INSERT INTO T_BF_MACRO_ECONOMIA (sg_uf, dt_referencia, nm_indicador, vl_indicador)
          VALUES (:1, :2, :3, :4)
      """
      cursor.executemany(sql_insert_macro, dados_pib)
      print(f"   -> {cursor.rowcount} pib inserido.")

      dados_ipca = df_ipca_final.values.tolist()
      sql_insert_macro = """
          INSERT INTO T_BF_MACRO_ECONOMIA (sg_uf, dt_referencia, nm_indicador, vl_indicador)
          VALUES (:1, :2, :3, :4)
      """
      cursor.executemany(sql_insert_macro, dados_ipca)
      print(f"   -> {cursor.rowcount} ipca inserido.")

      dados_pmc = df_pmc_melt_final.values.tolist()
      sql_insert_macro = """
          INSERT INTO T_BF_MACRO_ECONOMIA (sg_uf, dt_referencia, nm_indicador, vl_indicador)
          VALUES (:1, :2, :3, :4)
      """
      cursor.executemany(sql_insert_macro, dados_pmc)
      print(f"   -> {cursor.rowcount} pmc inserido.")

      # Commit final
      connection.commit()
      print("\n✅ CARGA FINALIZADA COM SUCESSO!")

except oracledb.Error as e:
    print(f"Erro fatal na conexão ou execução: {e}")

Conectou
   -> 748 selic inserido.
   -> 135 pib inserido.
   -> 135 ipca inserido.
   -> 918 pmc inserido.

✅ CARGA FINALIZADA COM SUCESSO!
